In [ ]:
!pip install "shapely<2.0.0"
!pip install google-cloud-aiplatform --upgrade
!pip install testbook pytest --upgrade

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
import vertexai
from vertexai.language_models import TextGenerationModel

PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1" # @param {type:"string"}

# define project information manually if the above code didn't work
if PROJECT_ID == "(unset)":
  PROJECT_ID = "[your-project-id]" # @param {type:"string"}

print(PROJECT_ID)

vertexai.init(project=PROJECT_ID, location=LOCATION)
parameters = {
    "candidate_count": 1,
    "max_output_tokens": 1024,
    "temperature": 0.8,
    "top_p": 0.8,
    "top_k": 40
}
model = TextGenerationModel.from_pretrained("text-bison")

In [ ]:
def getPositiveOrNegative(prompt):
  response = model.predict(
    """Context: You look at messages and categorize them as
    positive, negative, or neutral.

  Output only Positive, Negative, or Neutral.

  Message: {0}.
  Output: """.format(prompt),
    **parameters
    )

  return response.text.strip()


In [ ]:
print(getPositiveOrNegative("Dinner was at noon."))

In [ ]:
import unittest

class TestPositiveOrNegative(unittest.TestCase):

    def test_getPositiveOrNegative1(self):
      response = getPositiveOrNegative("Dinner was Great")
      self.assertEqual(response, "Positive")

    def test_getPositiveOrNegative2(self):
      response = getPositiveOrNegative("That broccoli was undercooked and cold")
      self.assertEqual(response, "Negative")

    def test_getPositiveOrNegative3(self):
      response = getPositiveOrNegative("We went to the new italian place for dinner")
      self.assertEqual(response, "Neutral")

unittest.main(argv=[''], verbosity=2, exit=False)

In [ ]:
getPositiveOrNegative("We went to the new italian place for dinner")

In [ ]:
def writeTweet(prompt):
  response = model.predict(
    """Context: You write Tweets for the Marketing Department at Luigi's Italian Cafe.

    1. Keep your Tweets below 100 characters
    2. Include the hashtag #EatAtLuigis at the end of every tweet

  Input: Write me a for our All You Can eat Spaghetti Wednesdays deal.
  Output: Spaghetti Wednesday is here! Come get all you can eat spaghetti and meatballs for only $9.99! #EatAtLuigis

  Input: {0}
  Output: """.format(prompt),
    **parameters
    )

  return response.text.strip()

In [ ]:
def are_tweets_the_same(tweet1, tweet2):
  response = model.predict(
    """Compare the following Tweets. Are they fundamentally the same?

    Only return Yes or No

    Tweet 1: {0}
    Tweet 2: {1}
    Output:

 """.format(tweet1, tweet2),
    **parameters
    )

  return response.text.strip()

In [ ]:
print(writeTweet("Write a tweet about our half-price bottles of wine every Thursday"))

In [ ]:
import unittest

class TestTextGeneration(unittest.TestCase):

    def test_tweet_results_1(self):
      actual_result = writeTweet("Write a tweet about our half-price bottles of wine every Thursday")
      expected_result = """Thirsty Thursday is here! Come enjoy half-price bottles of
      wine every Thursday at Luigi's! #EatAtLuigis"""
      same = are_tweets_the_same(actual_result, expected_result)
      self.assertEqual(same, "Yes")

    def test_tweet_results_Should_fail(self):
      actual_result = writeTweet("Write a tweet about our half-price bottles of wine every Thursday")
      expected_result = """Halloween party at the community center! Dress up and come out for a
      night of fun, food, and games. #Halloween"""
      same = are_tweets_the_same(actual_result, expected_result)
      self.assertEqual(same, "Yes")


unittest.main(argv=[''], verbosity=2, exit=False)